In [1]:
!pip install gensim
!pip install nltk

In [55]:
import re  
import pandas as pd 
from time import time  
from collections import defaultdict 
import string 
import multiprocessing
import os
import gensim
import sklearn
from sklearn import linear_model
from collections import Counter
import numpy as np
import scipy
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, cohen_kappa_score, classification_report
from nltk.tokenize import word_tokenize
import pickle

# word2vec
from gensim.models import Word2Vec, KeyedVectors, FastText
from gensim.models.phrases import Phrases, Phraser
from sklearn.model_selection import train_test_split
import logging

import nltk
from nltk.stem import PorterStemmer
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [46]:
path =  "../../Data/train/df_us_train.pickle"
df_us_train = pickle.load(open(path, "rb"))

path =  "../../Data/train/df_es_train.pickle"
df_es_train = pickle.load(open(path, "rb"))

In [49]:
from collections import Counter

punctuation = string.punctuation + "«»“”‘’…—"

stopwords_spanish = pd.read_csv(
    'https://raw.githubusercontent.com/Alir3z4/stop-words/master/spanish.txt'
).values
stopwords_spanish = Counter(stopwords_spanish.flatten().tolist())

stopwords_english = pd.read_csv(
    'https://raw.githubusercontent.com/Alir3z4/stop-words/master/english.txt'
).values
stopwords_english = Counter(stopwords_english.flatten().tolist())

## Contructor de sentence-embeddings

In [70]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

### Word2vec

In [31]:
%%time
us_emoji_w2v = Word2Vec.load("us_emoji_w2v.model")

2022-06-30 12:35:01,722 : INFO : loading Word2Vec object from us_emoji_w2v.model
2022-06-30 12:35:01,733 : INFO : loading wv recursively from us_emoji_w2v.model.wv.* with mmap=None
2022-06-30 12:35:01,734 : INFO : setting ignored attribute cum_table to None
2022-06-30 12:35:01,758 : INFO : Word2Vec lifecycle event {'fname': 'us_emoji_w2v.model', 'datetime': '2022-06-30T12:35:01.758834', 'gensim': '4.2.0', 'python': '3.8.12 (default, Oct 12 2021, 03:01:40) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19044-SP0', 'event': 'loaded'}


Wall time: 38.9 ms


In [32]:
%%time
es_emoji_w2v = Word2Vec.load("es_emoji_w2v.model")

2022-06-30 12:35:03,378 : INFO : loading Word2Vec object from es_emoji_w2v.model
2022-06-30 12:35:03,390 : INFO : loading wv recursively from es_emoji_w2v.model.wv.* with mmap=None
2022-06-30 12:35:03,391 : INFO : setting ignored attribute cum_table to None
2022-06-30 12:35:03,395 : INFO : Word2Vec lifecycle event {'fname': 'es_emoji_w2v.model', 'datetime': '2022-06-30T12:35:03.395021', 'gensim': '4.2.0', 'python': '3.8.12 (default, Oct 12 2021, 03:01:40) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19044-SP0', 'event': 'loaded'}


Wall time: 18.9 ms


In [48]:
def simple_tokenizer(doc, stopwords, lower=False):
    if lower:
        tokenized_doc = doc.translate(str.maketrans(
            '', '', punctuation)).lower().split()

    tokenized_doc = doc.translate(str.maketrans('', '', punctuation)).split()

    tokenized_doc = [
        token for token in tokenized_doc if token.lower() not in stopwords
    ]
    return tokenized_doc

def sentence_embedding_from_w2v(doc, w2v):
    pre_tokens = simple_tokenizer(doc, stopwords_spanish, lower=False)
    sentence_vect = np.zeros(w2v.vector_size)
    m = 0
    for token in pre_tokens:
        if token in w2v.wv:
            sentence_vect += w2v.wv[token]
            m += 1
    sentence_vect *= 1/m if m>0 else 1
    return sentence_vect

In [38]:
%%time
us_train_w2v_sentence_embedding = pd.DataFrame(np.array(
    [ sentence_embedding_from_w2v(doc, us_emoji_w2v) for doc in df_us_train["text"]]
))
us_train_w2v_sentence_embedding["id"] = df_us_train["id"]
us_train_w2v_sentence_embedding["label"] = df_us_train["label"]

Wall time: 18.4 s


In [39]:
%%time
es_train_w2v_sentence_embedding = pd.DataFrame(np.array(
    [ sentence_embedding_from_w2v(doc, es_emoji_w2v) for doc in df_es_train["text"]]
))
es_train_w2v_sentence_embedding["id"] = df_es_train["id"]
es_train_w2v_sentence_embedding["label"] = df_es_train["label"]

Wall time: 2.97 s


In [41]:
%%time
X_us = us_train_w2v_sentence_embedding.drop(columns=["label", "id"])

Wall time: 383 ms


In [42]:
%%time
X_es = es_train_w2v_sentence_embedding.drop(columns=["label", "id"])

Wall time: 91.8 ms


In [43]:
%%time
n_labels_us = us_train_w2v_sentence_embedding["label"].unique().shape[0]

Wall time: 23 ms


In [44]:
%%time
n_labels_es = es_train_w2v_sentence_embedding["label"].unique().shape[0]

Wall time: 4.99 ms


In [45]:
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import v_measure_score
from sklearn.metrics.cluster import rand_score